# Using Knn Classifier to differentiate Water Mines from Rocks

## Introduction

- The Sonar is a system that uses sound waves to detect objects under the water. (oceanservice.noaa.gov) It is mostly used to locate underwater hazards for navigation, search and map objects on the seafloor such as shipwrecks, and map the seafloor itself. (oceanservice.noaa.gov) What is interesting about these sound wave signals, which we will be using in our project as well, is the fact that the frequency of the signals bouncing off metal cylinders vs. rocks are transmitted at different frequencies. These data were obtained via signals being used at different aspect angles, spanning 90 degrees for the cylinder and 180 degrees for the rock.
- In this project, we will be training the system, which will determine whether the solar system that is being bounced off, is from a metal cylinder, or a roughly cylindrical rock
- There are 60 columns of frequency ranging from 0.0-1.0 and each of the numbers represents the energy within a particular frequency band, integrated over a certain period of time. Each of these sets of frequencies is tied to a label which is either “M” for metal cylinder or “R” for roughly cylindrical rock. We will implement these data in our project to train the system to recognize whether the frequency is related to a metal cylinder or a roughly cylindrical rock.

In [1]:
# set up the environment
import random

import altair as alt
import pandas as pd
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)

from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

alt.data_transformers.disable_max_rows()


# Simplify working with large datasets in Altair
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

**Preliminary exploratory data analysis:**

- The data set was stored at Kaggle in .csv format. 
- To ensure this data is loadable regardless of users, GitHub is used to host this file. It is accessed by referring to the permanent link of the raw file.
- To read the data set, simply read in using pd.read_csv. The data is properly formatted using comma as delimiters. 

In [2]:
sonar_data = pd.read_csv ("https://raw.githubusercontent.com/OminiCarlos/DSCI100_Group_14_Proposal_Mine_Finder/main/sonar.all-data.csv")
sonar_data

,Freq_1,Freq_2,Freq_3,Freq_4,Freq_5,Freq_6,Freq_7,Freq_8,Freq_9,Freq_10,...,Freq_52,Freq_53,Freq_54,Freq_55,Freq_56,Freq_57,Freq_58,Freq_59,Freq_60,Label
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M


After a thorough inspection, we can say that the data are already in clean format. Each column is a variable, each row is one observation and each value is a cell. 

To understand the content of the data set, we first take a look at sample size. 

In [3]:
nb_observations = sonar_data.shape[0]
nb_observations

208

208 is not a very big sample size, but it's enough to train our model. 


Secondly, we investigate the number of samples in each class, to verify if the sample is balanced. Imbalanced sample will let the dominant class hold the majority vote when it's not supposed to. This will give unnecessary favor to the majority class and hinder the model's accuracy. 

In [4]:
sample_breakdown = sonar_data["Label"].value_counts(normalize = True)
sample_breakdown

M    0.533654
R    0.466346
Name: Label, dtype: float64

We have 53% mine sample and 46% rock samples. Since the sample size is roughly the same, it is safe to say the sample is balanced. Therefore, we do not need to resample our data set. 

Since all the data are of the same type, which is the frequency, and that all the signal are in the same unit, the data is naturally centered. Therefore, we do not need to standardize the data set.

Now that we have made sure the data set is legitimate, it's time to split the data into training and testing set.

In [5]:
# set random state
np.random.seed(1024)

sonar_data_train, sonar_data_test = train_test_split(
    sonar_data, train_size=0.75, stratify=sonar_data["Label"]
)

sonar_data_train

,Freq_1,Freq_2,Freq_3,Freq_4,Freq_5,Freq_6,Freq_7,Freq_8,Freq_9,Freq_10,...,Freq_52,Freq_53,Freq_54,Freq_55,Freq_56,Freq_57,Freq_58,Freq_59,Freq_60,Label
63,0.0067,0.0096,0.0024,0.0058,0.0197,0.0618,0.0432,0.0951,0.0836,0.1180,...,0.0048,0.0023,0.0020,0.0040,0.0019,0.0034,0.0034,0.0051,0.0031,R
87,0.0856,0.0454,0.0382,0.0203,0.0385,0.0534,0.2140,0.3110,0.2837,0.2751,...,0.0172,0.0138,0.0079,0.0037,0.0051,0.0258,0.0102,0.0037,0.0037,R
158,0.0107,0.0453,0.0289,0.0713,0.1075,0.1019,0.1606,0.2119,0.3061,0.2936,...,0.0164,0.0120,0.0113,0.0021,0.0097,0.0072,0.0060,0.0017,0.0036,M
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
91,0.0253,0.0808,0.0507,0.0244,0.1724,0.3823,0.3729,0.3583,0.3429,0.2197,...,0.0178,0.0073,0.0079,0.0038,0.0116,0.0033,0.0039,0.0081,0.0053,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,0.0459,0.0437,0.0347,0.0456,0.0067,0.0890,0.1798,0.1741,0.1598,0.1408,...,0.0067,0.0032,0.0109,0.0164,0.0151,0.0070,0.0085,0.0117,0.0056,R
200,0.0131,0.0387,0.0329,0.0078,0.0721,0.1341,0.1626,0.1902,0.2610,0.3193,...,0.0150,0.0076,0.0032,0.0037,0.0071,0.0040,0.0009,0.0015,0.0085,M
123,0.0270,0.0163,0.0341,0.0247,0.0822,0.1256,0.1323,0.1584,0.2017,0.2122,...,0.0189,0.0204,0.0085,0.0043,0.0092,0.0138,0.0094,0.0105,0.0093,M
130,0.0443,0.0446,0.0235,0.1008,0.2252,0.2611,0.2061,0.1668,0.1801,0.3083,...,0.0274,0.0205,0.0141,0.0185,0.0055,0.0045,0.0115,0.0152,0.0100,M


I will group my data into 2 sets: Mine and Rock, to see if there is a visible pattern with their sonar profile. First I will group my data set by labels, then I will aggregate my data by calculating the mean value of each frequency. After that, I will plot the sonar profile using bar plots for both signals, to compare the intensity at each frequency. 

In [6]:
snoar_data_agg = sonar_data_train.groupby(["Label"]).mean().reset_index()
snoar_data_agg

,Label,Freq_1,Freq_2,Freq_3,Freq_4,Freq_5,Freq_6,Freq_7,Freq_8,Freq_9,...,Freq_51,Freq_52,Freq_53,Freq_54,Freq_55,Freq_56,Freq_57,Freq_58,Freq_59,Freq_60
0,M,0.034478,0.046443,0.054616,0.068734,0.089508,0.114306,0.127623,0.149060,0.220994,...,0.019617,0.016706,0.011980,0.012099,0.009931,0.008971,0.007801,0.008966,0.008417,0.006922
1,R,0.023474,0.029532,0.035075,0.039905,0.062152,0.098152,0.117866,0.121126,0.140244,...,0.012119,0.010173,0.009723,0.008992,0.008041,0.007697,0.007526,0.006670,0.006537,0.006077


We can see that the signal of Mine is different from that of rocks. For example,  the intensity of Freq_9 of the Mine is significantly higher than that of the rock. Now it's time to visualize the data.

In [7]:
# sort columns
sorted_cols = ['Label'] + sorted(snoar_data_agg.columns[1:], key=lambda x: int(x.split('_')[1]))

# melt data into long format
sonar_agg_melt = snoar_data_agg.melt(id_vars = "Label",
                                     var_name="Frequency",
                                     value_name = "Intensity"
                                    )
sonar_agg_melt

,Label,Frequency,Intensity
0,M,Freq_1,0.034478
1,R,Freq_1,0.023474
2,M,Freq_2,0.046443
3,R,Freq_2,0.029532
4,M,Freq_3,0.054616
...,...,...,...
115,R,Freq_58,0.006670
116,M,Freq_59,0.008417
117,R,Freq_59,0.006537
118,M,Freq_60,0.006922


In [8]:
agg_bar_plot = alt.Chart(sonar_agg_melt).mark_bar().encode(
    x = alt.X("Frequency", sort=sorted_cols[1:]),
    y = alt.Y("Intensity"),
    color = "Label"
)

agg_bar_plot

alt.Chart(...)

As can be seen in the bar plot, the intensity of the rock signal is generally weaker than that of the mines. This difference is very obvious in the region between Frequency 18 -31. This pattern tells us it is possible to distinguish a rock and a mine with the sonar signal. 


## Methods:

### Data selected:

- We do not need all the 60 frequencies. The visualization shows that after frequency 49, the signal is not significant between classes. We will discard them to have a smaller data set, so we can train the model more efficient.
    
    
### Plan of action:

Using K-nearest neighbor classification algorithm

- Execute a cross-validation in Python to choose the number of neighbors with a K-nearest neighbors classifier.
- Combine preprocessing and model training using make_pipeline
- Construct the confusion matrix to evaluate the performance of the model. 


In [9]:
# Encode the labels ('M' for mines and 'R' for rocks)
le = LabelEncoder()
sonar_data['Label'] = le.fit_transform(sonar_data['Label'])
sonar_data['Label']

0      1
1      1
2      1
3      1
4      1
      ..
203    0
204    0
205    0
206    0
207    0
Name: Label, Length: 208, dtype: int64

In [10]:
# Separate predictors and labels
X = sonar_data.drop('Label', axis=1)
y = sonar_data['Label']

In [17]:
# Split the data into training and testing sets
np.random.seed(2023)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=1024)
display (X_train, X_test, y_train, y_test)

,Freq_1,Freq_2,Freq_3,Freq_4,Freq_5,Freq_6,Freq_7,Freq_8,Freq_9,Freq_10,...,Freq_51,Freq_52,Freq_53,Freq_54,Freq_55,Freq_56,Freq_57,Freq_58,Freq_59,Freq_60
63,0.0067,0.0096,0.0024,0.0058,0.0197,0.0618,0.0432,0.0951,0.0836,0.1180,...,0.0029,0.0048,0.0023,0.0020,0.0040,0.0019,0.0034,0.0034,0.0051,0.0031
87,0.0856,0.0454,0.0382,0.0203,0.0385,0.0534,0.2140,0.3110,0.2837,0.2751,...,0.0128,0.0172,0.0138,0.0079,0.0037,0.0051,0.0258,0.0102,0.0037,0.0037
158,0.0107,0.0453,0.0289,0.0713,0.1075,0.1019,0.1606,0.2119,0.3061,0.2936,...,0.0079,0.0164,0.0120,0.0113,0.0021,0.0097,0.0072,0.0060,0.0017,0.0036
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
91,0.0253,0.0808,0.0507,0.0244,0.1724,0.3823,0.3729,0.3583,0.3429,0.2197,...,0.0085,0.0178,0.0073,0.0079,0.0038,0.0116,0.0033,0.0039,0.0081,0.0053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,0.0459,0.0437,0.0347,0.0456,0.0067,0.0890,0.1798,0.1741,0.1598,0.1408,...,0.0121,0.0067,0.0032,0.0109,0.0164,0.0151,0.0070,0.0085,0.0117,0.0056
200,0.0131,0.0387,0.0329,0.0078,0.0721,0.1341,0.1626,0.1902,0.2610,0.3193,...,0.0137,0.0150,0.0076,0.0032,0.0037,0.0071,0.0040,0.0009,0.0015,0.0085
123,0.0270,0.0163,0.0341,0.0247,0.0822,0.1256,0.1323,0.1584,0.2017,0.2122,...,0.0197,0.0189,0.0204,0.0085,0.0043,0.0092,0.0138,0.0094,0.0105,0.0093
130,0.0443,0.0446,0.0235,0.1008,0.2252,0.2611,0.2061,0.1668,0.1801,0.3083,...,0.0248,0.0274,0.0205,0.0141,0.0185,0.0055,0.0045,0.0115,0.0152,0.0100


,Freq_1,Freq_2,Freq_3,Freq_4,Freq_5,Freq_6,Freq_7,Freq_8,Freq_9,Freq_10,...,Freq_51,Freq_52,Freq_53,Freq_54,Freq_55,Freq_56,Freq_57,Freq_58,Freq_59,Freq_60
145,0.0721,0.1574,0.1112,0.1085,0.0666,0.1800,0.1108,0.2794,0.1408,0.0795,...,0.0234,0.0309,0.0127,0.0082,0.0281,0.0117,0.0092,0.0147,0.0157,0.0129
134,0.1083,0.1070,0.0257,0.0837,0.0748,0.1125,0.3322,0.4590,0.5526,0.5966,...,0.0172,0.0180,0.0110,0.0234,0.0276,0.0032,0.0084,0.0122,0.0082,0.0143
177,0.0201,0.0165,0.0344,0.0330,0.0397,0.0443,0.0684,0.0903,0.1739,0.2571,...,0.0125,0.0054,0.0057,0.0137,0.0109,0.0035,0.0056,0.0105,0.0082,0.0036
31,0.0084,0.0153,0.0291,0.0432,0.0951,0.0752,0.0414,0.0259,0.0692,0.1753,...,0.0056,0.0236,0.0114,0.0136,0.0117,0.0060,0.0058,0.0031,0.0072,0.0045
15,0.0298,0.0615,0.0650,0.0921,0.1615,0.2294,0.2176,0.2033,0.1459,0.0852,...,0.0154,0.0031,0.0153,0.0071,0.0212,0.0076,0.0152,0.0049,0.0200,0.0073
147,0.0654,0.0649,0.0737,0.1132,0.2482,0.1257,0.1797,0.0989,0.2460,0.3422,...,0.0243,0.0210,0.0361,0.0239,0.0447,0.0394,0.0355,0.0440,0.0243,0.0098
14,0.0124,0.0433,0.0604,0.0449,0.0597,0.0355,0.0531,0.0343,0.1052,0.2120,...,0.0078,0.0083,0.0057,0.0174,0.0188,0.0054,0.0114,0.0196,0.0147,0.0062
106,0.0331,0.0423,0.0474,0.0818,0.0835,0.0756,0.0374,0.0961,0.0548,0.0193,...,0.0133,0.0078,0.0174,0.0176,0.0038,0.0129,0.0066,0.0044,0.0134,0.0092
207,0.0260,0.0363,0.0136,0.0272,0.0214,0.0338,0.0655,0.1400,0.1843,0.2354,...,0.0181,0.0146,0.0129,0.0047,0.0039,0.0061,0.0040,0.0036,0.0061,0.0115
16,0.0352,0.0116,0.0191,0.0469,0.0737,0.1185,0.1683,0.1541,0.1466,0.2912,...,0.0426,0.0346,0.0158,0.0154,0.0109,0.0048,0.0095,0.0015,0.0073,0.0067


63     1
87     1
158    0
1      1
91     1
      ..
93     1
200    0
123    0
130    0
82     1
Name: Label, Length: 156, dtype: int64

145    0
134    0
177    0
31     1
15     1
147    0
14     1
106    0
207    0
16     1
205    0
65     1
56     1
114    0
203    0
198    0
174    0
77     1
18     1
187    0
120    0
186    0
170    0
46     1
185    0
164    0
115    0
39     1
34     1
94     1
40     1
162    0
193    0
51     1
32     1
140    0
90     1
42     1
84     1
66     1
116    0
124    0
64     1
9      1
85     1
194    0
62     1
156    0
110    0
129    0
166    0
28     1
Name: Label, dtype: int64

Now it's the time to search for the best parameter K for our Knn model. To do this, we need to first specify the parameters of the SearchGridCV function. 

In [42]:
# Specify the estimator. 
# Since the data is naturally centered, we don't need to standardize it
knn_spec =  KNeighborsClassifier()
display (knn_spec.get_params()) # get the parameter that specifies the No. of neighbors.

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

- We will create a search grid by plugging K = 2 to 20 in the 'n_neighbors' parameter. 
- Then we will perform grid search with 5-fold cross-validation to find the best value for k (number of neighbors).
- Here we choose **recall** as our score, because we want to have as little false negative for our model as possible. The reason is that we water mines are dangerous. The water mines that we fail to detect could potentially cause damage to ships and even cost lives. On the other hand, when we have false positive, we risk wasting peoples time to verify if the object is really a mine, which is acceptable. 
- Lastly, we will report the performance of the model in a dataframe.

In [43]:
#set the parameter grid.
param_grid = {
    'n_neighbors': range(2, 21)
}

# Perform grid search with 5-fold cross-validation to find the best value for k (number of neighbors)
mine_detector_grid = GridSearchCV(estimator = knn_spec,
                                  param_grid = param_grid,
                                  cv=5,
                                  return_train_score=True,
                                  scoring='recall')

mine_detector_grid.fit(X_train, y_train)

CV_results = pd.DataFrame(mine_detector_grid.fit(X_train,y_train).cv_results_)
CV_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.002961,0.000287,0.005482,0.000268,2,{'n_neighbors': 2},0.600000,0.666667,0.533333,0.428571,...,0.574286,0.085799,3,0.793103,0.758621,0.689655,0.694915,0.694915,0.726242,0.042000
1,0.003063,0.000366,0.005760,0.000511,3,{'n_neighbors': 3},0.733333,0.733333,0.733333,0.785714,...,0.768571,0.048711,1,0.844828,0.862069,0.827586,0.830508,0.813559,0.835710,0.016502
2,0.003101,0.000145,0.005861,0.000436,4,{'n_neighbors': 4},0.600000,0.533333,0.533333,0.428571,...,0.547619,0.072656,4,0.741379,0.758621,0.706897,0.661017,0.610169,0.695617,0.054202
3,0.003046,0.000035,0.005935,0.000623,5,{'n_neighbors': 5},0.666667,0.600000,0.666667,0.428571,...,0.629524,0.116969,2,0.793103,0.793103,0.741379,0.728814,0.661017,0.743483,0.048877
4,0.003256,0.000396,0.006114,0.001022,6,{'n_neighbors': 6},0.533333,0.400000,0.333333,0.357143,...,0.410476,0.069752,19,0.672414,0.672414,0.517241,0.576271,0.491525,0.585973,0.075740
5,0.003158,0.000325,0.005338,0.000166,7,{'n_neighbors': 7},0.600000,0.666667,0.533333,0.357143,...,0.517143,0.112187,6,0.775862,0.706897,0.551724,0.627119,0.576271,0.647575,0.083267
6,0.002908,0.000097,0.005430,0.000229,8,{'n_neighbors': 8},0.466667,0.600000,0.466667,0.357143,...,0.463810,0.078973,13,0.568966,0.534483,0.482759,0.508475,0.474576,0.513852,0.034645
7,0.003109,0.000338,0.005764,0.000547,9,{'n_neighbors': 9},0.533333,0.800000,0.466667,0.357143,...,0.517143,0.152488,6,0.586207,0.568966,0.534483,0.661017,0.593220,0.588778,0.041437
8,0.002995,0.000416,0.005485,0.000491,10,{'n_neighbors': 10},0.466667,0.733333,0.400000,0.357143,...,0.462857,0.141049,14,0.517241,0.465517,0.534483,0.525424,0.491525,0.506838,0.025139
9,0.002797,0.000118,0.005484,0.000131,11,{'n_neighbors': 11},0.600000,0.800000,0.533333,0.357143,...,0.529524,0.165873,5,0.586207,0.465517,0.568966,0.593220,0.542373,0.551257,0.046317


With the result in hand, we visualize the model's performance with different K in a line chart, to help us find the best K. 

In [41]:
recall_plot = alt.Chart(CV_results).mark_line(point=True).encode(
    x = alt.X("param_n_neighbors"),
    y = alt.Y("mean_test_score")
)
recall_plot 

alt.Chart(...)

It seems we have the highest recall when K = 3. We will choose this to train our model! Other K value gives us a recall significantly less than 70%, which is not acceptable. For K = 2 and 4, the result is a little more than 60%, which is just slightly better than making the decision by flipping a coin.

In [47]:
# Set the random seed.
np.random.seed(2023)

mine_detector_spec = KNeighborsClassifier( n_neighbors = 3)
mine_detector_fit = mine_detector_spec.fit (X_train,y_train)

The model is trained. Now we are going to use the test data set to evaluate the model's performance. We will compare the prediction and the true value side-by-side.

In [52]:
Mine_detector_reality_check = pd.DataFrame({"True Label": y_test,
                                           "Prediction": mine_detector_fit .predict(X_test)})
Mine_detector_reality_check

,True Label,Prediction
145,0,1
134,0,0
177,0,1
31,1,1
15,1,1
147,0,0
14,1,1
106,0,0
207,0,1
16,1,1


In many instances the model seems to give an incorrect labeling. We will take a closer look by constructing a confusion matrix. 

In [54]:
mnist_conf_mat = pd.crosstab(Mine_detector_reality_check["True Label"],
                             Mine_detector_reality_check["Prediction"])

mnist_conf_mat

Prediction,0,1
True Label,,
0,22,6
1,5,19


Recall that 0 is Rock and 1 is Mine, we move on to report the recall and accuracy of our model.

$\mathrm{precision} = \frac{\mathrm{number \; of  \; correct \; positive \; predictions}}{\mathrm{total \;  number \;  of \; positive  \; predictions}}.$

$\mathrm{recall} = \frac{\mathrm{number \; of  \; correct  \; positive \; predictions}}{\mathrm{total \;  number \;  of  \; positive \; test \; set \; observations}}$


In [63]:
Mine_precision= 19/ (6+19)
Mine_precision

0.76

In [64]:
Mine_recall = 19/ (5+19)
Mine_recall

0.7916666666666666

## Discussion

With the analysis, we have successfully trained a Knn-model that can distinguish water mine from rocks based on the signal of a sonar system. We have determined that for the sake of public safety, recall should be favored when training the model. With a Gridsearch, we conclude that the model produces the highest recall by choose K = 3. 

## For Winnie and Farbod

- discuss whether this is what you expected to find?
    We expect to use our classifier to predict whether the object underwater is an explosive mine or a rock.
- discuss what impact could such findings have?
    This model allows us to automate the water mine detection. With such a model, underwater robots can locate and flag mines automatically. They can search through a large area of water thoroughly. It is hard for humans because it is expensive to train and hire human analysts. This can save the lives of civilians, especially the fishermen. On the other hand, it is also important for national defense because it can protect the expensive battleships.
    
- discuss what future questions could this lead to?
    This analysis could lead to future questions such as whether there is a way to train the model with fewer parameters and, if so, how to find them, whether the model can detect mines of other shapes, and whether we can develop a model that can label the type of the mine.
